Objet : EGMS — extraction zone + décomposition ASC/DESC + cumuls + extrêmes + export TS

Ce notebook produit, pour une zone d’intérêt (AOI) définie par un polygone KML + l’emprise du bâtiment, une extraction EGMS L2a et une décomposition ASC/DESC permettant d’estimer deux composantes de vitesse :

Vv : verticale (mm/an)

Ve : Est–Ouest (mm/an)

Entrées

ixi_rennes.kml : polygone AOI (WGS84) utilisé pour contraindre le bord Sud.

building_roi.gpkg : empreinte bâtiment (EPSG:2154 attendu) utilisée pour N/E/W.

geographic_extents_nicol.gpkg : index des tuiles EGMS (EPSG:3035).

Répertoire EGMS local (base_dir) contenant les CSV tuiles L2a.

Sorties (dossier egms_extract_zone/)

pids/ : PIDs extraits par tuile dans l’AOI (chunked).

attr/ : attributs EGMS filtrés par PID (chunked).

figs/ : cartes (Vv, Ve, cumuls ΔV, ΔE) + carte+TS des extrêmes.

outputs.gpkg :

vel_mm_per_year : points “paires” ASC/DESC avec Vv, Ve, cond, dist_m

cum_mm : cumuls ΔV, ΔE (approx = V × durée)

building_roi : empreinte bâtiment

Méthode (résumé)

AOI hybride : bbox avec Sud issu du KML, et N/E/W issus du bâtiment reprojeté en EPSG:3035.

Sélection des tuiles : intersection bbox AOI vs extents EGMS.

Extraction PIDs : lecture CSV par chunks, filtre bbox puis filtre polygonal.

Extraction attributs : pour chaque tuile, lecture par chunks + filtre sur set de PIDs.

Pairing ASC/DESC : jointure nearest (max_distance=5 m) puis réduction 1–1 par distance minimale.

Décomposition : résolution d’un système 2x2 (LOS ASC/DESC → Ve, Vv) avec filtre conditionnement cond <= 50.

Cumuls : ΔV = Vv × durée, ΔE = Ve × durée (durées paramétrables).

Analyse extrêmes : repérage du point de cumul vertical max(|ΔV|) par période + export des TS LOS (ASC/DESC) associées, avec superposition chaînée par offset médian sur recouvrements.

Paramètres clés à vérifier selon site

MAX_DIST_M (par défaut 5 m), COND_MAX (50)

DURATION_Y (7 ans / 5 ans ici) : à ajuster si besoin selon bornes exactes

orbit_from_tile_safe() : mapping tuile → ASC/DESC (spécifique au cas)

Unités des séries temporelles exportées : ici on exporte les colonnes dates du CSV EGMS (LOS), à documenter selon produit.

In [ ]:
import os, glob
import xml.etree.ElementTree as ET

import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import contextily as ctx

from shapely.geometry import Polygon, Point, box
from shapely.prepared import prep
from pyproj import Transformer
from geopandas import sjoin_nearest

# ============================================================
# 0) PARAMÈTRES
# ============================================================
VERBOSE = True

# --- Inputs ---
KML_PATH = "ixi_rennes.kml"
BUILDING_GPKG = "building_roi.gpkg"  # CRS 2154 attendu
EXTENTS_GPKG_NAME = "geographic_extents_nicol.gpkg"

BASE_DIR_CANDIDATES = [
    "/mnt/c/Users/DataScience/OneDrive - jll.spear/Documents/EGMS/",
    "/mnt/c/Users/nicol/OneDrive - jll.spear/Documents/EGMS/",
]

# --- Outputs ---
OUT_DIR = "egms_extract_zone"
OUT_PIDS = os.path.join(OUT_DIR, "pids")
OUT_ATTR = os.path.join(OUT_DIR, "attr")
OUT_FIGS = os.path.join(OUT_DIR, "figs")
OUT_GPKG = os.path.join(OUT_DIR, "outputs.gpkg")

# --- Processing ---
CHUNKSIZE = 200_000
PERIODS = ["2015_2021", "2018_2022", "2019_2023"]

# pairing ASC/DESC
MAX_DIST_M = 5.0
COND_MAX = 50.0

# cumuls
DURATION_Y = {
    "2015_2021": 7.0,  # ajuste si besoin (années pleines vs bornes exactes)
    "2018_2022": 5.0,
    "2019_2023": 5.0,
}

# ------------------------------------------------------------
# Mapping "tile_safe -> orbit/period" (à adapter si besoin)
# ------------------------------------------------------------
def orbit_from_tile_safe(safe: str) -> str:
    if "030_0283" in safe or "030_0284" in safe:
        return "ASC"
    if "081_0789" in safe or "154_0787" in safe:
        return "DESC"
    return "UNK"

def period_from_tile_safe(safe: str) -> str:
    if "2018_2022" in safe:
        return "2018_2022"
    if "2019_2023" in safe:
        return "2019_2023"
    return "2015_2021"


# ============================================================
# 1) UTILITAIRES (prints + checks)
# ============================================================
def vprint(*args):
    if VERBOSE:
        print(*args)

def ensure_dir(path):
    os.makedirs(path, exist_ok=True)
    vprint("[DIR]", path)

def assert_exists(path, label="file"):
    if not os.path.exists(path):
        raise FileNotFoundError(f"Missing {label}: {path}")
    vprint("[OK]", label, "exists:", path)

def summarize_gdf(gdf, name="gdf"):
    vprint(f"\n[{name}] n={len(gdf)} cols={list(gdf.columns)}")
    vprint(f"[{name}] crs={gdf.crs}")
    try:
        vprint(f"[{name}] bounds={gdf.total_bounds}")
    except Exception:
        pass

def summarize_df(df, name="df", head=3):
    vprint(f"\n[{name}] shape={df.shape}")
    vprint(f"[{name}] columns={list(df.columns)}")
    vprint(df.head(head))


# ============================================================
# 2) AOI : KML (sud) + bâtiment (N/E/W)
# ============================================================
def find_base_dir(candidates):
    for p in candidates:
        if os.path.exists(p):
            vprint("[BASE_DIR]", p)
            return p
    raise FileNotFoundError("No EGMS base_dir found in BASE_DIR_CANDIDATES.")

def parse_kml_polygon_wgs84(kml_path) -> Polygon:
    ns = {"kml": "http://www.opengis.net/kml/2.2"}
    root = ET.parse(kml_path).getroot()
    node = root.find(".//kml:Polygon/kml:outerBoundaryIs/kml:LinearRing/kml:coordinates", ns)
    if node is None or node.text is None:
        raise ValueError("No Polygon coordinates found in KML.")
    coords = []
    for token in node.text.strip().split():
        lon, lat = token.split(",")[:2]
        coords.append((float(lon), float(lat)))
    poly = Polygon(coords)
    if not poly.is_valid:
        vprint("[WARN] KML polygon invalid -> buffer(0) fix attempt")
        poly = poly.buffer(0)
    return poly

def project_polygon(poly4326: Polygon, dst_epsg=3035) -> Polygon:
    tr = Transformer.from_crs("EPSG:4326", f"EPSG:{dst_epsg}", always_xy=True)
    xs, ys = poly4326.exterior.coords.xy
    X, Y = tr.transform(xs, ys)
    poly = Polygon(zip(X, Y))
    if not poly.is_valid:
        vprint("[WARN] projected polygon invalid -> buffer(0) fix attempt")
        poly = poly.buffer(0)
    return poly

def build_hybrid_aoi_bbox_3035(building_gpkg, kml_path):
    assert_exists(building_gpkg, "BUILDING_GPKG")
    assert_exists(kml_path, "KML_PATH")

    # KML -> 3035
    poly_kml_3035 = project_polygon(parse_kml_polygon_wgs84(kml_path), 3035)
    xmin_k, ymin_k, xmax_k, ymax_k = poly_kml_3035.bounds
    vprint("[AOI] KML_3035 bounds:", (xmin_k, ymin_k, xmax_k, ymax_k))

    # building -> 3035
    b = gpd.read_file(building_gpkg)
    summarize_gdf(b, "building_raw")
    b3035 = b.to_crs(3035)
    summarize_gdf(b3035, "building_3035")
    xmin_b, ymin_b, xmax_b, ymax_b = b3035.total_bounds
    vprint("[AOI] BUILDING_3035 bounds:", (xmin_b, ymin_b, xmax_b, ymax_b))

    # AOI bbox : S = KML ; N/E/W = building
    poly3035 = box(xmin_b, ymin_k, xmax_b, ymax_b)

    vprint("[AOI] hybrid bbox3035:", poly3035.bounds)
    vprint("[AOI] hybrid area (m²):", poly3035.area)

    bbox3035 = box(*poly3035.bounds)
    return b3035, poly3035, bbox3035, prep(poly3035)


# ============================================================
# 3) Sélection des tuiles EGMS qui intersectent bbox AOI
# ============================================================
def select_tiles(extents_gpkg_path, bbox3035):
    assert_exists(extents_gpkg_path, "EXTENTS_GPKG")
    ext = gpd.read_file(extents_gpkg_path)
    summarize_gdf(ext, "extents_raw")

    # important: extents en général déjà en 3035 (à vérifier)
    vprint("[EXTENTS] CRS:", ext.crs)

    tile_boxes = gpd.GeoSeries(
        [box(r["Min Easting"], r["Min Northing"], r["Max Easting"], r["Max Northing"]) for _, r in ext.iterrows()],
        crs=ext.crs
    )
    ext = ext.assign(_tile_geom=tile_boxes)
    sel = ext[ext["_tile_geom"].intersects(bbox3035)].drop(columns=["_tile_geom"]).copy()

    vprint("[TILES] intersect AOI:", len(sel), "/", len(ext))
    return sel


# ============================================================
# 4) Extraction PID dans AOI par tuile (chunk + bbox + polygon)
# ============================================================
def extract_zone_pids(big_csv_path, out_csv, bbox3035, poly3035_prepared, chunksize=200_000):
    usecols = ["pid", "easting", "northing"]
    dtypes  = {"pid": "string", "easting": "float64", "northing": "float64"}

    if os.path.exists(out_csv):
        os.remove(out_csv)

    xmin, ymin, xmax, ymax = bbox3035.bounds
    first, kept, read_total = True, 0, 0

    for chunk in pd.read_csv(big_csv_path, usecols=usecols, dtype=dtypes, chunksize=chunksize):
        read_total += len(chunk)

        m = (chunk["easting"].between(xmin, xmax) & chunk["northing"].between(ymin, ymax))
        sub = chunk.loc[m]
        if sub.empty:
            continue

        keep = [poly3035_prepared.contains(Point(x, y)) for x, y in zip(sub["easting"].to_numpy(), sub["northing"].to_numpy())]
        sub = sub.loc[keep]
        if sub.empty:
            continue

        sub.to_csv(out_csv, mode="w" if first else "a", index=False, header=first)
        first = False
        kept += len(sub)

    return kept, read_total


# ============================================================
# 5) Extraction attributs pour PID (chunk + filtre set)
# ============================================================
def extract_attrs_for_pids(big_csv_path, pids_csv_path, out_attr_csv, chunksize=200_000):
    pids = set(pd.read_csv(pids_csv_path, usecols=["pid"])["pid"].astype(str))
    if not pids:
        return 0, 0

    usecols = ["pid","easting","northing","los_east","los_up","mean_velocity","temporal_coherence"]

    if os.path.exists(out_attr_csv):
        os.remove(out_attr_csv)

    first, kept, read_total = True, 0, 0
    for chunk in pd.read_csv(big_csv_path, usecols=usecols, chunksize=chunksize):
        read_total += len(chunk)
        chunk["pid"] = chunk["pid"].astype(str)
        sub = chunk[chunk["pid"].isin(pids)]
        if sub.empty:
            continue

        sub.to_csv(out_attr_csv, mode="w" if first else "a", index=False, header=first)
        first = False
        kept += len(sub)

    return kept, read_total


# ============================================================
# 6) Décomposition ASC/DESC : solve 2x2 -> Ve, Vv + cond filter
# ============================================================
def decompose_asc_desc(pairs, cond_max=50.0):
    Ve = np.full(len(pairs), np.nan, float)
    Vv = np.full(len(pairs), np.nan, float)
    cond = np.full(len(pairs), np.nan, float)

    for i, row in enumerate(pairs.itertuples(index=False)):
        A = np.array([[row.los_east_left,  row.los_up_left],
                      [row.los_east_right, row.los_up_right]], dtype=float)
        b = np.array([row.mean_velocity_left, row.mean_velocity_right], dtype=float)

        if not (np.isfinite(A).all() and np.isfinite(b).all()):
            continue

        c = np.linalg.cond(A)
        cond[i] = c
        if (not np.isfinite(c)) or (c > cond_max):
            continue

        try:
            sol = np.linalg.solve(A, b)  # [Ve, Vv]
            Ve[i], Vv[i] = sol[0], sol[1]
        except np.linalg.LinAlgError:
            pass

    out = pairs.copy()
    out["V_E"] = Ve
    out["V_V"] = Vv
    out["cond"] = cond
    return out


# ============================================================
# 7) Plot panels basemap (unique échelle symétrique)
# ============================================================
def sym_norm(series, p=(2, 98)):
    s = pd.to_numeric(series, errors="coerce")
    lo, hi = np.nanpercentile(s, p)
    vabs = max(abs(lo), abs(hi))
    return mcolors.TwoSlopeNorm(vmin=-vabs, vcenter=0, vmax=vabs), vabs

def add_basemap(ax, zoom=19, alpha=0.25):
    ctx.add_basemap(
        ax,
        source=ctx.providers.CartoDB.Positron,
        zoom=min(zoom, 20),
        alpha=alpha,
        attribution=False,
        reset_extent=False,
        zorder=0
    )

def plot_panels_map(
    gdf_3857,
    gdf_building_3857,
    var,
    unit,
    title,
    periods=("2015_2021", "2018_2022", "2019_2023"),
    pclip=(2, 98),
    zoom=19,
    point_size=45,
    point_alpha=0.9,
    pad_m=10,
    out_png=None,
):
    norm, vabs = sym_norm(gdf_3857[var], pclip)

    xmin, ymin, xmax, ymax = gdf_building_3857.total_bounds
    xmin -= pad_m; ymin -= pad_m; xmax += pad_m; ymax += pad_m

    fig, axes = plt.subplots(1, len(periods), figsize=(21, 8), constrained_layout=True)
    sc = None

    for ax, per in zip(axes, periods):
        sub = gdf_3857[gdf_3857["period"] == per]

        ax.set_xlim(xmin, xmax)
        ax.set_ylim(ymin, ymax)
        ax.set_aspect("equal")
        ax.set_axis_off()

        add_basemap(ax, zoom=zoom, alpha=0.25)

        gdf_building_3857.boundary.plot(ax=ax, color="black", linewidth=1.2, zorder=2)

        sc = ax.scatter(
            sub.geometry.x,
            sub.geometry.y,
            c=sub[var],
            cmap="coolwarm",
            norm=norm,
            s=point_size,
            alpha=point_alpha,
            edgecolor="black",
            linewidth=0.25,
            zorder=5
        )

        ax.set_title(per, fontsize=16)

        vprint(f"[PLOT] {var} {per}: n={len(sub)}  nan={sub[var].isna().sum()}")

    cbar = fig.colorbar(sc, ax=axes, shrink=0.85, pad=0.02)
    cbar.set_label(f"{var} ({unit}) |max|≈{vabs:.1f}", fontsize=14)

    fig.suptitle(title, fontsize=20)

    if out_png:
        ensure_dir(os.path.dirname(out_png) or ".")
        fig.savefig(out_png, dpi=200, bbox_inches="tight")
        vprint("[SAVE]", out_png)

    plt.show()


# ============================================================
# 8) PIPELINE
# ============================================================
# --- dirs
ensure_dir(OUT_DIR)
ensure_dir(OUT_PIDS)
ensure_dir(OUT_ATTR)
ensure_dir(OUT_FIGS)

# --- base dir + extents
base_dir = find_base_dir(BASE_DIR_CANDIDATES)
extents_path = os.path.join(base_dir, EXTENTS_GPKG_NAME)
assert_exists(extents_path, "EXTENTS_GPKG")

# --- AOI
b3035, poly3035, bbox3035, poly3035_prep = build_hybrid_aoi_bbox_3035(BUILDING_GPKG, KML_PATH)

# --- tiles
tiles = select_tiles(extents_path, bbox3035)
tiles_l2a = tiles[tiles["File Identifier"].astype(str).str.contains("L2a", na=False)].copy()

vprint("\n[TILES] total intersect:", len(tiles), "| L2a:", len(tiles_l2a))
vprint(tiles_l2a["File Identifier"].head(10).to_list())

# --- 1) PID extraction per tile
for fid in tiles_l2a["File Identifier"].tolist():
    csv_path = os.path.join(base_dir, fid + ".csv")
    safe = fid.replace("/", "_")
    out_pids = os.path.join(OUT_PIDS, f"{safe}_PIDS.csv")

    if not os.path.exists(csv_path):
        vprint("[MISS CSV]", fid)
        continue

    if os.path.exists(out_pids):
        vprint("[SKIP PIDS]", safe)
        continue

    n_kept, n_read = extract_zone_pids(csv_path, out_pids, bbox3035, poly3035_prep, CHUNKSIZE)
    vprint(f"[PIDS] {safe}: kept={n_kept} (read={n_read}) -> {out_pids}")

# --- 2) ATTR extraction per tile (based on its PID list)
for fid in tiles_l2a["File Identifier"].tolist():
    csv_path = os.path.join(base_dir, fid + ".csv")
    safe = fid.replace("/", "_")

    pids_csv = os.path.join(OUT_PIDS, f"{safe}_PIDS.csv")
    out_attr = os.path.join(OUT_ATTR, f"{safe}_ATTR.csv")

    if not os.path.exists(csv_path):
        continue
    if not os.path.exists(pids_csv):
        vprint("[NO PIDS]", safe)
        continue
    if os.path.exists(out_attr):
        vprint("[SKIP ATTR]", safe)
        continue

    n_kept, n_read = extract_attrs_for_pids(csv_path, pids_csv, out_attr, CHUNKSIZE)
    vprint(f"[ATTR] {safe}: kept={n_kept} (read={n_read}) -> {out_attr}")

# --- 3) Load ATTR -> gdf_l2a_3035
attr_paths = sorted(glob.glob(os.path.join(OUT_ATTR, "*_ATTR.csv")))
vprint("\n[ATTR] files:", len(attr_paths))
if len(attr_paths) == 0:
    raise RuntimeError("No ATTR files produced. Check AOI, extents, paths, or CSV availability.")

dfs = []
for p in attr_paths:
    safe = os.path.basename(p).replace("_ATTR.csv", "")
    df = pd.read_csv(p)
    df["tile_safe"] = safe
    df["period"] = period_from_tile_safe(safe)
    df["orbit"]  = orbit_from_tile_safe(safe)
    dfs.append(df)

df_l2a = pd.concat(dfs, ignore_index=True)
summarize_df(df_l2a, "df_l2a")

gdf_l2a_3035 = gpd.GeoDataFrame(
    df_l2a,
    geometry=gpd.points_from_xy(df_l2a["easting"], df_l2a["northing"]),
    crs="EPSG:3035"
)
summarize_gdf(gdf_l2a_3035, "gdf_l2a_3035")

vprint("\n[COUNTS] by period/orbit:")
vprint(gdf_l2a_3035.groupby(["period","orbit"]).size())

# --- 4) Pair + decompose per period -> concat gdf_vel
gdf_vel_list = []

for period in PERIODS:
    sub  = gdf_l2a_3035[gdf_l2a_3035["period"] == period]
    asc  = sub[sub["orbit"] == "ASC"]
    desc = sub[sub["orbit"] == "DESC"]

    vprint(f"\n=== PERIOD {period} === ASC={len(asc)}  DESC={len(desc)}")
    if len(asc) == 0 or len(desc) == 0:
        vprint("[WARN] missing ASC or DESC -> skip", period)
        continue

    pairs = sjoin_nearest(
        asc[["pid","mean_velocity","los_east","los_up","geometry"]],
        desc[["pid","mean_velocity","los_east","los_up","geometry"]],
        how="inner",
        max_distance=MAX_DIST_M,
        distance_col="dist_m",
    )
    vprint("[PAIRS] raw:", len(pairs))

    # enforce 1-1: keep smallest dist both sides
    pairs = pairs.sort_values("dist_m").drop_duplicates(subset=["pid_right"], keep="first")
    pairs = pairs.sort_values("dist_m").drop_duplicates(subset=["pid_left"], keep="first")
    vprint("[PAIRS] 1-1:", len(pairs), "| dist(min/med/max) =",
           float(pairs["dist_m"].min()), float(pairs["dist_m"].median()), float(pairs["dist_m"].max()))

    pairs = decompose_asc_desc(pairs, cond_max=COND_MAX)

    vprint("[DECOMP] nan(V_V) =", int(pairs["V_V"].isna().sum()),
           "| nan(V_E) =", int(pairs["V_E"].isna().sum()),
           "| cond>max =", int((pairs["cond"] > COND_MAX).sum()))

    gdfv = gpd.GeoDataFrame(
        pairs[["pid_left","pid_right","dist_m","V_E","V_V","cond","geometry"]],
        geometry="geometry",
        crs="EPSG:3035"
    ).rename(columns={"pid_left":"pid_asc","pid_right":"pid_desc"})
    gdfv["period"] = period
    gdf_vel_list.append(gdfv)

if len(gdf_vel_list) == 0:
    raise RuntimeError("No period produced pairs. Check MAX_DIST_M, orbit mapping, AOI content.")

gdf_vel = pd.concat(gdf_vel_list, ignore_index=True)
summarize_gdf(gdf_vel, "gdf_vel")

# --- 5) Cumuls
gdf_cum_all = gdf_vel.copy()
gdf_cum_all["duration_y"] = gdf_cum_all["period"].map(DURATION_Y).astype(float)
gdf_cum_all["dV_mm"] = gdf_cum_all["V_V"].astype(float) * gdf_cum_all["duration_y"]
gdf_cum_all["dE_mm"] = gdf_cum_all["V_E"].astype(float) * gdf_cum_all["duration_y"]

vprint("\n[CUM] duration_y counts:")
vprint(gdf_cum_all["duration_y"].value_counts(dropna=False))

# --- 6) Reproject once for plots
gdf_vel_3857 = gdf_vel.to_crs(3857)
gdf_cum_all_3857 = gdf_cum_all.to_crs(3857)
gdf_building_3857 = b3035.to_crs(3857)

# --- 7) Plots + exports
plot_panels_map(
    gdf_vel_3857, gdf_building_3857,
    var="V_V", unit="mm/an",
    title="Vitesse verticale Vv (mm/an) — décomposition ASC/DESC",
    out_png=os.path.join(OUT_FIGS, "Vv_mm_per_year_panels.png"),
    point_size=45,
)

plot_panels_map(
    gdf_vel_3857, gdf_building_3857,
    var="V_E", unit="mm/an",
    title="Vitesse Est–Ouest Ve (mm/an) — décomposition ASC/DESC",
    out_png=os.path.join(OUT_FIGS, "Ve_mm_per_year_panels.png"),
    point_size=45,
)

plot_panels_map(
    gdf_cum_all_3857, gdf_building_3857,
    var="dV_mm", unit="mm",
    title="Déplacement cumulé vertical ΔV (mm) — approx Vv × durée",
    out_png=os.path.join(OUT_FIGS, "Cum_Vert_mm_panels_basemap.png"),
    point_size=60,
)

plot_panels_map(
    gdf_cum_all_3857, gdf_building_3857,
    var="dE_mm", unit="mm",
    title="Déplacement cumulé Est–Ouest ΔE (mm) — approx Ve × durée",
    out_png=os.path.join(OUT_FIGS, "Cum_East_mm_panels_basemap.png"),
    point_size=60,
)

# --- 8) Exports GPKG
ensure_dir(os.path.dirname(OUT_GPKG) or ".")
gdf_vel.to_file(OUT_GPKG, layer="vel_mm_per_year", driver="GPKG")
gdf_cum_all.to_file(OUT_GPKG, layer="cum_mm", driver="GPKG")
b3035.to_file(OUT_GPKG, layer="building_roi", driver="GPKG")

vprint("\n[EXPORT] written:", OUT_GPKG)


# Plot Map + Séries Temporelles 

In [ ]:
# ============================================================
# 9) EXTREMES + EXPORT TS + FIGURE MAP+TS (à coller après pipeline)
# Dépend de: gdf_vel, gdf_building_3857, tiles_l2a, base_dir
# ============================================================



# -----------------------------
# Params
# -----------------------------
PERIODS        = ["2015_2021", "2018_2022", "2019_2023"]
PERIOD_ORDER   = ["2015_2021", "2018_2022", "2019_2023"]
PERIOD_COLORS  = {"2015_2021":"#1f77b4", "2018_2022":"#ff7f0e", "2019_2023":"#2ca02c"}
DURATION_Y     = {"2015_2021": 7.0, "2018_2022": 5.0, "2019_2023": 5.0}

ZOOM   = 20
MAP_PT_SIZE = 35
MAP_PT_EDGE = 0.2
MAP_PAD     = 25

OUT_TS_EXT = "egms_extract_zone_ts_extremes"
os.makedirs(OUT_TS_EXT, exist_ok=True)
TS_CSV = os.path.join(OUT_TS_EXT, "ts_extremes_long.csv")

# Colonne à analyser (cumul vertical)
VAR = "cum_Vv_mm"


# -----------------------------
# Helpers
# -----------------------------
def period_from_fid(fid: str) -> str:
    fid = str(fid)
    if "2018_2022" in fid: return "2018_2022"
    if "2019_2023" in fid: return "2019_2023"
    return "2015_2021"

def get_extreme_row(gdf, period, var=VAR):
    sub = gdf[gdf["period"] == period].dropna(subset=[var, "pid_asc", "pid_desc"])
    if sub.empty:
        return None
    return sub.loc[sub[var].abs().idxmax()]

def sym_norm(series, p=(2,98)):
    series = pd.to_numeric(series, errors="coerce")
    lo, hi = np.nanpercentile(series, p)
    vabs = max(abs(lo), abs(hi))
    return mcolors.TwoSlopeNorm(vmin=-vabs, vcenter=0, vmax=vabs), vabs

def get_date_cols(cols):
    return [c for c in cols if isinstance(c, str) and c.isdigit() and len(c) == 8]

def export_extreme_ts_all_tiles(tiles_l2a, base_dir, pids_ext, out_csv_path, chunksize=200_000):
    """
    Exporte les TS LOS des PIDs extrêmes, pour toutes les tuiles (toutes périodes),
    en format long: pid, date, value, fid.
    """
    pids_ext = set(map(str, pids_ext))

    if os.path.exists(out_csv_path):
        os.remove(out_csv_path)

    first_write = True
    total_rows = 0

    for fid in tiles_l2a["File Identifier"].tolist():
        csv_path = os.path.join(base_dir, fid + ".csv")
        if not os.path.exists(csv_path):
            print("[MISS]", fid)
            continue

        head = pd.read_csv(csv_path, nrows=1)
        date_cols = get_date_cols(head.columns)
        if not date_cols:
            print("[NO DATE COLS]", fid)
            continue

        usecols = ["pid"] + date_cols
        kept_tile = 0

        for chunk in pd.read_csv(csv_path, usecols=usecols, chunksize=chunksize):
            chunk["pid"] = chunk["pid"].astype(str)
            chunk = chunk[chunk["pid"].isin(pids_ext)]
            if chunk.empty:
                continue

            df_long = chunk.melt(
                id_vars=["pid"],
                value_vars=date_cols,
                var_name="date",
                value_name="value"
            )

            df_long["date"]  = pd.to_datetime(df_long["date"], format="%Y%m%d", errors="coerce")
            df_long["value"] = pd.to_numeric(df_long["value"], errors="coerce")
            df_long = df_long.dropna(subset=["date","value"])

            df_long["fid"] = fid

            df_long.to_csv(
                out_csv_path,
                mode="w" if first_write else "a",
                index=False,
                header=first_write
            )
            first_write = False

            kept_tile += len(df_long)
            total_rows += len(df_long)

        print("[TS]", fid, "->", kept_tile)

    print("DONE. total rows:", total_rows)
    print("Saved:", out_csv_path)
    return total_rows


def chained_offsets(df_pid, period_order=PERIOD_ORDER, min_overlap=3):
    """
    Chaînage simple: p0 ref, p1 alignée sur p0, p2 alignée sur p1, etc.
    """
    offsets = {period_order[0]: 0.0}
    for p_prev, p_cur in zip(period_order[:-1], period_order[1:]):
        d_prev = df_pid[df_pid["period"] == p_prev][["date","value"]]
        d_cur  = df_pid[df_pid["period"] == p_cur ][["date","value"]]

        overlap = np.intersect1d(d_prev["date"].to_numpy(), d_cur["date"].to_numpy())
        if len(overlap) < min_overlap:
            offsets[p_cur] = offsets[p_prev]
            continue

        v_prev = d_prev[d_prev["date"].isin(overlap)]["value"].to_numpy()
        v_cur  = d_cur [d_cur ["date"].isin(overlap)]["value"].to_numpy()
        delta  = np.median(v_prev - v_cur)

        offsets[p_cur] = offsets[p_prev] + delta

    return offsets

def compute_global_ylim(df_ts, pids, period_order=PERIOD_ORDER, min_overlap=3):
    ymin, ymax = np.inf, -np.inf
    for pid in pids:
        sub = df_ts[df_ts["pid"] == pid]
        if sub.empty:
            continue
        off = chained_offsets(sub, period_order=period_order, min_overlap=min_overlap)
        for p in period_order:
            sp = sub[sub["period"] == p]
            if sp.empty:
                continue
            v = sp["value"].to_numpy() + off.get(p, 0.0)
            ymin = min(ymin, np.nanmin(v))
            ymax = max(ymax, np.nanmax(v))

    if not np.isfinite(ymin) or not np.isfinite(ymax):
        return None
    pad = 0.05 * (ymax - ymin if ymax > ymin else 1.0)
    return (ymin - pad, ymax + pad)

def plot_pid_chain(ax, df_ts, pid, title,
                   period_order=PERIOD_ORDER,
                   xlim=None, ylim=None,
                   show_legend=True,
                   show_xticks=True):

    sub = df_ts[df_ts["pid"] == pid].copy()
    if sub.empty:
        ax.text(0.5,0.5,"No TS",ha="center",va="center",transform=ax.transAxes)
        return

    off = chained_offsets(sub, period_order=period_order, min_overlap=3)

    for p in period_order:
        sp = sub[sub["period"] == p].sort_values("date")
        if sp.empty:
            continue
        ax.plot(sp["date"], sp["value"] + off[p],
                lw=1.2, color=PERIOD_COLORS[p], label=p)

    ax.text(0.01, 0.02, title,
            transform=ax.transAxes, ha="left", va="bottom",
            fontsize=11,
            bbox=dict(boxstyle="round,pad=0.2",
                      facecolor="white", alpha=0.7, linewidth=0))

    ax.set_ylabel("LOS (mm)")
    ax.grid(True, ls="--", alpha=0.3)

    if show_legend:
        ax.legend(fontsize=8, loc="upper right")

    if xlim is not None: ax.set_xlim(*xlim)
    if ylim is not None: ax.set_ylim(*ylim)

    if not show_xticks:
        ax.tick_params(axis="x", which="both", labelbottom=False)


# -----------------------------
# 1) Construire la variable extrême sur gdf_vel
# -----------------------------
gdf_ext = gdf_vel.copy()
gdf_ext["duration_y"] = gdf_ext["period"].map(DURATION_Y).astype(float)
gdf_ext[VAR] = gdf_ext["V_V"].astype(float) * gdf_ext["duration_y"]

# CRS carte
gdf_3857 = gdf_ext.to_crs(3857) if (gdf_ext.crs and gdf_ext.crs.to_epsg() != 3857) else gdf_ext
b_3857 = gdf_building_3857

# -----------------------------
# 2) Trouver PIDs extrêmes (ASC/DESC) par période
# -----------------------------
ext_rows = {}
pids_ext = set()

for per in PERIODS:
    row = get_extreme_row(gdf_3857, per, var=VAR)
    if row is None:
        print("[WARN] no extreme row:", per)
        continue
    ext_rows[per] = row
    pids_ext.add(str(row["pid_asc"]))
    pids_ext.add(str(row["pid_desc"]))

print("Extreme PIDs:", sorted(pids_ext))

# -----------------------------
# 3) Export TS des PIDs extrêmes (si pas déjà fait)
# -----------------------------
if not os.path.exists(TS_CSV):
    n = export_extreme_ts_all_tiles(
        tiles_l2a=tiles_l2a,
        base_dir=base_dir,
        pids_ext=pids_ext,
        out_csv_path=TS_CSV,
        chunksize=200_000
    )
    print("TS rows:", n)
else:
    print("[SKIP] TS already exists:", TS_CSV)

# -----------------------------
# 4) Charger TS + xlim/ylim global
# -----------------------------
df_ts = pd.read_csv(TS_CSV)
df_ts["date"] = pd.to_datetime(df_ts["date"])
df_ts["pid"]  = df_ts["pid"].astype(str)
df_ts["period"] = df_ts["fid"].apply(period_from_fid)

xmin = df_ts["date"].min()
xmax = df_ts["date"].max()
xlim_global = (xmin - pd.DateOffset(months=6), xmax + pd.DateOffset(months=6))

ylim_global = compute_global_ylim(df_ts, pids_ext)

# -----------------------------
# 5) Figure finale
# -----------------------------
def plot_complex(periods=PERIODS, var=VAR, zoom=ZOOM,
                 xlim_ts=xlim_global, ylim_ts=ylim_global):

    norm, vabs = sym_norm(gdf_3857[var], p=(2,98))

    fig = plt.figure(figsize=(18, 12))
    gs = fig.add_gridspec(
        nrows=len(periods), ncols=2,
        width_ratios=[1.0, 1.6],
        hspace=0.35, wspace=0.15
    )

    sc_for_cbar = None

    for i, per in enumerate(periods):
        r = ext_rows.get(per)
        if r is None:
            continue

        pid_a = str(r["pid_asc"])
        pid_d = str(r["pid_desc"])

        # ---- MAP (gauche)
        axm = fig.add_subplot(gs[i, 0])
        sub = gdf_3857[gdf_3857["period"] == per]

        b_3857.plot(ax=axm, facecolor="none", edgecolor="black", linewidth=1.2)
        sc = axm.scatter(
            sub.geometry.x, sub.geometry.y,
            c=sub[var], cmap="coolwarm", norm=norm,
            s=MAP_PT_SIZE, edgecolor="black", linewidth=MAP_PT_EDGE
        )
        if sc_for_cbar is None:
            sc_for_cbar = sc

        axm.scatter([r.geometry.x], [r.geometry.y],
                    s=220, facecolor="none", edgecolor="black", linewidth=2)

        axm.set_title(f"{per} — EXT | {var}={float(r[var]):.1f} mm")
        axm.set_axis_off()
        axm.set_aspect("equal")

        xminb, yminb, xmaxb, ymaxb = b_3857.total_bounds
        axm.set_xlim(xminb - MAP_PAD, xmaxb + MAP_PAD)
        axm.set_ylim(yminb - MAP_PAD, ymaxb + MAP_PAD)

        ctx.add_basemap(axm, source=ctx.providers.CartoDB.Positron,
                        zoom=zoom, attribution=False)

        # ---- TS panel (droite) : 2 sous-axes
        axt = fig.add_subplot(gs[i, 1])
        pos = axt.get_position()
        axt.remove()

        ax1 = fig.add_axes([pos.x0, pos.y0 + pos.height*0.52, pos.width, pos.height*0.48])
        ax2 = fig.add_axes([pos.x0, pos.y0,                 pos.width, pos.height*0.48])

        plot_pid_chain(ax1, df_ts, pid_a, f"ASC {pid_a}",
                       xlim=xlim_ts, ylim=ylim_ts, show_legend=True, show_xticks=False)
        plot_pid_chain(ax2, df_ts, pid_d, f"DESC {pid_d}",
                       xlim=xlim_ts, ylim=ylim_ts, show_legend=True, show_xticks=True)
        ax2.set_xlabel("Date")

    if sc_for_cbar is not None:
        cbar = fig.colorbar(sc_for_cbar, ax=fig.axes, shrink=0.6, pad=0.04, fraction=0.03)
        cbar.set_label(f"{var} (mm)  |max|≈{vabs:.1f}")

    plt.suptitle("Extremes cum_Vv_mm — map + TS ASC/DESC (superposition chainée)", y=0.995)
    plt.show()

plot_complex()
